In [1]:
%matplotlib widget

from matplotlib import pyplot as plt
import os
from themachinethatgoesping.echosounders.simrad import FileRaw,SimradDatagram_type_from_string,datagram_type_to_string,t_SimradDatagramType
import themachinethatgoesping.echosounders.simrad.datagrams as dg
import themachinethatgoesping.tools.progressbars as prg
import numpy
import os, sys
from tqdm.auto import tqdm
from collections import defaultdict
import numpy as np

#load files
folders =  ["/home/data/"]
#folders =  ["/home/data/turbeams/"]

files = []
for folder in folders:
    for r,d,f in os.walk(folder):
        for file in f:
            if file.endswith('.raw'):
                files.append(r + '/' +file)
            
files.sort()
#files
file_name = files[0]

In [2]:
#as headers

file_name = files[0]

file_size = os.stat(file_name).st_size

last_pos=0

packages = defaultdict(int)



with open(file_name,'rb') as ifi:
    with tqdm(total=file_size) as pbar:
        while ifi.tell() < file_size:
            b1 = ifi.read(4)
            length1 = int.from_bytes(b1,'little')
            b2 = ifi.read(length1)
            b3 = ifi.read(4)
            length2 = int.from_bytes(b3,'little')
            b = bytearray(b1) + bytearray(b2) + bytearray(b3)
            header = dg.SimradDatagram.from_binary(bytes(b),False)
            
            packages[header.datagram_type] = header._raw_DatagramType

            pbar.update(ifi.tell()-last_pos)
            last_pos = ifi.tell()
            
pbar.close()           

for k,v in packages.items():
    print(k,':',v)


  0%|          | 0/524344788 [00:00<?, ?it/s]

t_SimradDatagramType.XML0 : 810306904
t_SimradDatagramType.FIL1 : 827083078
t_SimradDatagramType.NME0 : 809848142
t_SimradDatagramType.MRU0 : 810897997
t_SimradDatagramType.RAW3 : 861356370


In [ ]:
#as unknown
file_name = files[0]

file_size = os.stat(file_name).st_size

last_pos=0

packages = defaultdict(list)



with open(file_name,'rb') as ifi:
    with tqdm(total=file_size) as pbar:
        while ifi.tell() < file_size:
            b1 = ifi.read(4)
            length1 = int.from_bytes(b1,'little')
            b2 = ifi.read(length1)
            b3 = ifi.read(4)
            length2 = int.from_bytes(b3,'little')
            b = bytearray(b1) + bytearray(b2) + bytearray(b3)
            datagram = dg.SimradUnknown.from_binary(bytes(b),True)
            
            packages[datagram.datagram_type].append(datagram)

            pbar.update(ifi.tell()-last_pos)
            last_pos = ifi.tell()
            
pbar.close()           

In [ ]:
for k,v in packages.items():
    print(k,len(v))

In [ ]:
packages[t_SimradDatagramType.XML0][1000].raw_content